In [0]:
import nltk
import pickle

In [48]:
nltk.download(['averaged_perceptron_tagger','punkt'])

[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [0]:
from nltk.tokenize import word_tokenize
import random

In [0]:
allowedWordTypes = ['J','R','V']  # J - adjective, R - adverb, V - verb

In [0]:
short_pos = open('positive.txt','r').read()
short_neg = open('negetive.txt','r').read()
document = []
allWords = []

In [0]:
for p in short_pos.split('\n'):
  document.append((p,"pos"))
  words = word_tokenize(p)
  pos = nltk.pos_tag(words)
  for w in pos:
    #if w[1][0] in allowedWordTypes:
      allWords.append(w[0].lower())

In [53]:
print(len(document))

5180


In [54]:
print(document[0])

('the rock is destined to be the 21st century\'s new " conan " and that he\'s going to make a splash even greater than arnold schwarzenegger , jean-claud van damme or steven segal . ', 'pos')


In [0]:
for p in short_neg.split('\n'):
  document.append((p,"neg"))
  words = word_tokenize(p)
  pos = nltk.pos_tag(words)
  for w in pos:
   # if w[1][0] in allowedWordTypes:
      allWords.append(w[0].lower())

In [56]:
len(document)

10511

In [57]:
print(document[5181])

("it's so laddish and juvenile , only teenage boys could possibly find it funny . ", 'neg')


In [0]:
random.shuffle(document)

In [59]:
document[0],document[1]

(("assayas' ambitious , sometimes beautiful adaptation of jacques chardonne's novel . ",
  'pos'),
 ('a lame comedy . ', 'neg'))

In [0]:
fe = nltk.FreqDist(allWords)

In [0]:
features_select = list(fe.keys())[:5000]

In [62]:
len(features_select)

5000

In [0]:
def feature_extract(doc):
  features = {}
  wordInDoc = word_tokenize(doc)
  for word in features_select:
    features[word] = (word in wordInDoc)
  return features

In [0]:
feature_vectors = []
for (rev,category) in document:
  feature_vectors.append((feature_extract(rev),category))

In [0]:
f_f = open('features.pickle','wb')
pickle.dump(feature_vectors,f_f)
f_f.close()

In [0]:
f_fs = open('features_select.pickle','wb')
pickle.dump(features_select,f_fs)
f_fs.close()

In [0]:
feature_vectors = pickle.load(open('features.pickle','rb'))

In [0]:
training_set = feature_vectors[:10000]
testing_set = feature_vectors[10000:]

In [0]:
model = nltk.NaiveBayesClassifier.train(training_set)
nltk.classify.accuracy(model,testing_set)

0.7592954990215264

#Start here if pickle files are not present

In [0]:
from nltk.classify.scikitlearn import SklearnClassifier

In [0]:
from sklearn.linear_model import SGDClassifier, LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.naive_bayes import MultinomialNB, BernoulliNB

In [0]:
model_sgd = SklearnClassifier(SGDClassifier())
model_sgd.train(feature_vectors)
f_sgd = open('sgd.pickle','wb')
pickle.dump(model_sgd,f_sgd)
f_sgd.close()

In [0]:
model_rf = SklearnClassifier(RandomForestClassifier())
model_rf.train(feature_vectors)
f_rf = open('rf.pickle','wb')
pickle.dump(model_rf,f_rf)
f_rf.close()

In [0]:
model_lr = SklearnClassifier(LogisticRegression(max_iter=1000))
model_lr.train(feature_vectors)
f_lr = open('lr.pickle','wb')
pickle.dump(model_lr,f_lr)
f_lr.close()

In [0]:
model_nb = SklearnClassifier(MultinomialNB())
model_nb.train(feature_vectors)
f_nb = open('nb.pickle','wb')
pickle.dump(model_nb,f_nb)
f_nb.close()

In [0]:
model_bb = SklearnClassifier(BernoulliNB())
model_bb.train(feature_vectors)
f_bb = open('bb.pickle','wb')
pickle.dump(model_bb,f_bb)
f_bb.close()

In [0]:
from nltk.classify import ClassifierI
from statistics import mode

In [0]:
class VoteClassifier(ClassifierI):
  def __init__(self,*classifiers):
    self._classifiers = classifiers
  def classify(self,features):
    votes = []
    for classifier in self._classifiers:
      out = classifier.classify(features)
      votes.append(out)
    return mode(votes)
  def confidence(self,features):
    votes = []
    for classifier in self._classifiers:
      out = classifier.classify(features)
      votes.append(out)
    count_vectors = votes.count(mode(votes))
    return count_vectors / len(votes)

In [0]:
vc = VoteClassifier(model_sgd,model_rf,model_lr,model_nb,model_bb)

In [0]:
nltk.classify.accuracy(vc,feature_vectors[10000:])

0.9119373776908023

In [0]:
f_vc = open('vc.pickle','wb')
pickle.dump(vc,f_vc)
f_vc.close()

In [0]:
from sklearn.neural_network import MLPClassifier

In [0]:
model_nn = SklearnClassifier(MLPClassifier(hidden_layer_sizes=(5000,100,2),max_iter=10,verbose=1))

In [84]:
model_nn.train(training_set)

Iteration 1, loss = 0.63688555
Iteration 2, loss = 0.48382166
Iteration 3, loss = 0.36435220
Iteration 4, loss = 0.29681592
Iteration 5, loss = 0.26177666
Iteration 6, loss = 0.24505452
Iteration 7, loss = 0.23306005
Iteration 8, loss = 0.22415389
Iteration 9, loss = 0.21548182
Iteration 10, loss = 0.20957386


/usr/local/lib/python3.6/dist-packages/sklearn/neural_network/_multilayer_perceptron.py:571: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (10) reached and the optimization hasn't converged yet.
  % self.max_iter, ConvergenceWarning)


<SklearnClassifier(MLPClassifier(activation='relu', alpha=0.0001, batch_size='auto', beta_1=0.9,
              beta_2=0.999, early_stopping=False, epsilon=1e-08,
              hidden_layer_sizes=(5000, 100, 2), learning_rate='constant',
              learning_rate_init=0.001, max_fun=15000, max_iter=10,
              momentum=0.9, n_iter_no_change=10, nesterovs_momentum=True,
              power_t=0.5, random_state=None, shuffle=True, solver='adam',
              tol=0.0001, validation_fraction=0.1, verbose=1, warm_start=False))>

In [85]:
nltk.classify.accuracy(model_nn,testing_set)

0.7377690802348337

#Start here if already file present

In [0]:
feature_select = pickle.load(open('features_select.pickle','rb'))

In [0]:
vc = pickle.load(open('vc.pickle','rb'))

In [0]:
def sentiment(doc):
  features = feature_extract(doc)
  return vc.classify(features), vc.confidence(features)

In [31]:
sentiment("The Movie was awesome!! The acting was great and plot was wonderful")

('pos', 1.0)

In [34]:
sentiment('I donot like this movie')

('neg', 1.0)

In [36]:
sentiment('I read that thing! it was 2/10')

('neg', 1.0)